In [3]:
import os
import sys
spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")

import findspark
findspark.init()
import pyspark

In [4]:
number_cores = 6
memory_gb = 16
conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set('spark.driver.memory', '{}g'.format(memory_gb)))
sc = pyspark.SparkContext(conf=conf)

In [5]:
!dir /users/trush/CSC496/PageRank

data  spark-3.ipynb


In [6]:
! cat /users/trush/CSC496/PageRank/data/small_graph.dat

y y
y a
a y
a m
m a

In [7]:
graph_data = sc.textFile("/users/trush/CSC496/PageRank/data/small_graph.dat")
graph_data.take(10)

['y y', 'y a', 'a y', 'a m', 'm a']

In [8]:
# Incoming and outgoing links in a page
links = graph_data.map(lambda line: (line.split(" ")[0], line.split(" ")[1])) \
        .groupByKey() \
        .mapValues(list)
links.take(10)

[('y', ['y', 'a']), ('a', ['y', 'm']), ('m', ['a'])]

In [9]:
N = links.count()
ranks = links.map(lambda line: (line[0], 1/N))
ranks.take(N)

[('y', 0.3333333333333333),
 ('a', 0.3333333333333333),
 ('m', 0.3333333333333333)]

In [10]:
votes = ranks.join(links)
votes.take(N)

[('y', (0.3333333333333333, ['y', 'a'])),
 ('a', (0.3333333333333333, ['y', 'm'])),
 ('m', (0.3333333333333333, ['a']))]

In [12]:
def calculateVotes(t):
    res = []
    for item in t[1][1]:
        count = len(t[1][1])
        res.append((item, t[1][0]/count))
    return res
# Weight of votes from original source
calculateVotes(('y', (0.3333333333333333, ['y', 'a'])))

[('y', 0.16666666666666666), ('a', 0.16666666666666666)]

In [13]:
votes = ranks.join(links) \
        .flatMap(calculateVotes)
votes.collect()

[('y', 0.16666666666666666),
 ('a', 0.16666666666666666),
 ('y', 0.16666666666666666),
 ('m', 0.16666666666666666),
 ('a', 0.3333333333333333)]

In [15]:
ranks.collect()

[('y', 0.3333333333333333), ('a', 0.5), ('m', 0.16666666666666666)]

In [16]:
ranks = votes.reduceByKey(lambda x, y: x + y)
ranks.collect()

[('y', 0.3333333333333333), ('a', 0.5), ('m', 0.16666666666666666)]